# 📘 RAG Application (DMP Example) using LangChain + **Local Llama (Ollama)** + FAISS


✅ We use:
- **Ollama** to run a local Llama model for generation (e.g., `llama3.2`, `llama3.1`, etc.)
- **Ollama embeddings** for vector search (e.g., `nomic-embed-text` or `mxbai-embed-large`)
- **FAISS** as the vector store

You will:
1. Use couple of PDFs
2. Load it
3. Chunk it
4. Embed + index with FAISS
5. Build:
   - **Baseline (no RAG)**
   - **RAG (retrieve + grounded answer)**
6. Compare results for the same DMP question

---
## 🔴 Important
You must have **Ollama installed and running** on your machine.

- Install Ollama: https://ollama.com
- Then pull models from the terminal (or from notebook cells below):
  - `ollama pull llama3.2`
  - `ollama pull nomic-embed-text`  *(embedding model)*

## 🔧 1) Install Python dependencies (run once)
! pip install -r requirements.txt

## 🦙 2) Make sure Ollama is installed + pull models

Run these in your terminal **once** (recommended):
```bash
ollama pull llama3.2
ollama pull nomic-embed-text
```

Or you can try running the commands below from the notebook.  
(If your environment doesn't allow shell commands, do it in the terminal.)

In [16]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


# --- Choose models you have pulled via `ollama pull ...`
LLM_MODEL = "llama3.2"            # generation model
EMBED_MODEL = "nomic-embed-text"  # embeddings model (recommended)

# Initialize Ollama chat model
llm = ChatOllama(model=LLM_MODEL, temperature=0.2)

# Initialize Ollama embeddings model
embeddings = OllamaEmbeddings(model=EMBED_MODEL)

print("✅ Ollama models configured:", LLM_MODEL, "|", EMBED_MODEL)

✅ Ollama models configured: llama3.2 | nomic-embed-text


## 📄 3) Load Load the document using LangChain

For a blog/tutorial example, we use a couple of NIH DMS plan pdf.
In production, you would replace this with:
- NIH policy pages (HTML/PDF)
- DMPTool guidance text
- Institutional sharing policy text
- Repository documentation

The key idea: **RAG retrieves from these sources** instead of the LLM guessing.


In [18]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
try:
    from langchain_ollama import ChatOllama
except Exception:
    from langchain_community.chat_models import ChatOllama


DATA_DIR = Path("data")
pdf_paths = sorted(DATA_DIR.glob("*.pdf"))

if not pdf_paths:
    raise FileNotFoundError("No PDFs found in ./data. Put NIH PDFs inside the data folder.")

docs = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(str(pdf_path))
    pdf_docs = loader.load()  # one Document per page
    for d in pdf_docs:
        d.metadata["source_file"] = pdf_path.name
        # PyPDFLoader usually sets page in metadata; keep it consistent:
        d.metadata["page"] = d.metadata.get("page", None)
    docs.extend(pdf_docs)

print(f"Loaded {len(pdf_paths)} PDF(s)")
print(f"Total page-docs: {len(docs)}")
print("Example metadata:", docs[0].metadata)
print("Example text (first 400 chars):", docs[0].page_content[:400])


Loaded 3 PDF(s)
Total page-docs: 9
Example metadata: {'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-07-08T17:11:53+00:00', 'moddate': '2025-07-08T17:11:53+00:00', 'source': 'data\\Human_4.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': 'Human_4.pdf'}
Example text (first 400 chars): Data Management and Sharing Plan 
Element 1: Data Type: 
A. Types and amount of scientific data expected to be generated in the project:  
Summarize the types and estimated amount of scientific data expected to be generated in the project. 
This project will perform secondary data analysis on kidney magnetic resonance imaging (MRI) data to 
determine the parenchymal kidney volume. Analysis will be


## ✂️ 4) Chunk the text

Why chunk?
- Embeddings work best on moderate-size text segments
- Retrieval becomes more precise

We use `RecursiveCharacterTextSplitter` to chunk with overlap.

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(docs)

# Add chunk_id for citations
for i, c in enumerate(chunks):
    c.metadata["chunk_id"] = f"chunk_{i:06d}"

print("Total chunks:", len(chunks))
print("Sample chunk metadata:", chunks[0].metadata)
print("Sample chunk text (first 300 chars):", chunks[0].page_content[:300])


Total chunks: 31
Sample chunk metadata: {'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-07-08T17:11:53+00:00', 'moddate': '2025-07-08T17:11:53+00:00', 'source': 'data\\Human_4.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': 'Human_4.pdf', 'chunk_id': 'chunk_000000'}
Sample chunk text (first 300 chars): Data Management and Sharing Plan 
Element 1: Data Type: 
A. Types and amount of scientific data expected to be generated in the project:  
Summarize the types and estimated amount of scientific data expected to be generated in the project. 
This project will perform secondary data analysis on kidney


## 🔍 5) Build FAISS vector store 

- Embed each chunk
- Store in FAISS

In [20]:
EMBED_MODEL = "nomic-embed-text"   # or "mxbai-embed-large"
BASE_URL = "http://localhost:11434"

embeddings = OllamaEmbeddings(model=EMBED_MODEL, base_url=BASE_URL)

# Build FAISS index
vectorstore = FAISS.from_documents(chunks, embeddings)

# Optional: save index locally
vectorstore.save_local("faiss_index_nih")
print("FAISS index built and saved to ./faiss_index_nih")


C:\Users\Nahid\AppData\Local\Temp\ipykernel_43760\3264666804.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=EMBED_MODEL, base_url=BASE_URL)


FAISS index built and saved to ./faiss_index_nih


## 🧠 6) Build retriever + helper functions

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

def format_docs_for_prompt(docs):
    """Create a compact context string + show citations info."""
    blocks = []
    for d in docs:
        src = d.metadata.get("source_file", "unknown")
        page = d.metadata.get("page", "NA")
        cid = d.metadata.get("chunk_id", "NA")
        text = d.page_content.strip()
        blocks.append(f"[{cid} | {src} | page={page}]\n{text}")
    return "\n\n---\n\n".join(blocks)

def extract_citations(text):
    """Find chunk citations like chunk_000123 in the answer."""
    return sorted(set(re.findall(r"chunk_\d{6}", text)))


## 🧠 7) Build Baseline vs RAG pipelines

We’ll do it in a simple, explicit way (easy to understand for a blog post).

### Baseline (no RAG)
- Just ask the Llama model directly

### RAG
- Retrieve top-k relevant chunks
- Inject them into the prompt
- Ask model to cite chunk IDs


## 7.1 Baseline (NO RAG) answer function

In [22]:
LLM_MODEL = "llama3.2"

llm = ChatOllama(model=LLM_MODEL, base_url=BASE_URL, temperature=0.2)

def answer_without_rag(question: str) -> str:
    prompt = f"""You are helping write an NIH Data Management and Sharing (DMS) Plan.

Answer the question clearly and concisely.

Question:
{question}

Return:
- Answer: (paragraph)
- Notes: (bullets, if needed)
"""
    return llm.invoke(prompt).content


## 7.2) RAG answer function

In [25]:
def answer_with_rag(question: str, k: int = 5) -> dict:
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    # ✅ New LangChain API
    retrieved = retriever.invoke(question)

    context = format_docs_for_prompt(retrieved)

    prompt = f"""You are helping write an NIH Data Management and Sharing (DMS) Plan.

Use ONLY the context below to answer. If the context is insufficient, say what is missing.

Context:
{context}

Question:
{question}

Return exactly:
- Answer: (paragraph)
- Key points: (bullets)
- Citations: (list chunk_ids you relied on, like chunk_000123)
"""
    out = llm.invoke(prompt).content
    cited = extract_citations(out)

    return {
        "answer": out,
        "retrieved_docs": retrieved,
        "retrieved_chunk_ids": [d.metadata.get("chunk_id") for d in retrieved],
        "cited_chunk_ids": cited
    }


## 8) Quick test with one question

In [33]:
question = (
    "From the PDF DMS plan example, extract the EXACT: "
    "(1) repository name used for deposit, "
    "(2) when data will be released (timing language), "
    "(3) how long data remains available, "
    "(4) what access model is used (e.g., tiered access) and what agreement/contract is required. "
    "Return bullets and include short quotes."
)

print("==== NO RAG ====")
print(answer_without_rag(question))

print("\n==== WITH RAG ====")
rag = answer_with_rag(question, k=5)
print(rag["answer"])

print("\n---- RAG Evidence (Retrieved Chunks) ----")
for d in rag["retrieved_docs"]:
    print(f"\n[{d.metadata.get('chunk_id')} | {d.metadata.get('source_file')} | page={d.metadata.get('page')}]")
    print(d.page_content[:700])

print("\nRetrieved chunk ids:", rag["retrieved_chunk_ids"])
print("Cited chunk ids:", rag["cited_chunk_ids"])

==== NO RAG ====
Here are the extracted information from the PDF DMS plan example:

* Repository name used for deposit: "National Center for Biotechnology Information (NCBI) Gene Expression Omnibus (GEO)"
* When data will be released: "Data will be made available to the public 12 months after publication of the final manuscript."
* How long data remains available: "Data will remain available on GEO for a minimum of 10 years from the date of deposit, and will be periodically reviewed and updated as necessary."
* Access model used: "Tiered access"
* Agreement/contract required: "Researchers must agree to comply with the terms of the NIH Data Sharing Policy and the NCBI Data Sharing Agreement."

Note: The exact wording may vary depending on the specific DMS plan example, but these are the extracted information based on a typical example.

==== WITH RAG ====
Here are the extracted answers:

- Answer: 
The data will be made available through the dbGaP following standard review and release p